In [1]:
import os

from openai import OpenAI

In [2]:
client = OpenAI()

### Basic

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You're a helpful assitant."},
        {"role": "user", "content": "what is a linter in vscode and how it is diff from formatter?"
        }
    ]
)

In [ ]:
response = completion.choices[0].message.content
print(response)

### Structured Output

In [ ]:
from pydantic import BaseModel

class MyEvents(BaseModel):
    name: str
    date: str
    participants: list[str]

In [ ]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "Extract the event information"},
        {
            "role": "user", "content": "Suman and Gita are going to a movie on Friday."
        },
    ],
    response_format=MyEvents
)

In [ ]:
event = completion.choices[0].message.parsed

In [ ]:
print(type(event))

In [ ]:
print(event)

### Function Calling

In [3]:
import requests
from dotenv import load_dotenv

_ = load_dotenv()

WEATHER_API = os.getenv('WEATHER_API')


def get_weather(city: str):
    """This is a public API to fetch the weather of given city."""
    
    
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API}&units=metric'

    response = requests.get(url)
    return response.json()['main']['temp']

get_weather('Kathmandu')

18.87

In [4]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

In [ ]:
# weather_tool = [
#     {
#         "type": "function",
#         "function": {
#             "name": "get_weather",
#             "description": "Get a current temprature for proivded city",
#             "parameters": {
#                 "type": "object",
#                 "properties": {
#                     "city": {"type": "string"}
#                     },
#                 "required": "city",
#                 "additionalProperties": False,
#             },
#             "strict": True
#         }
#     }
# ]

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string"},
                    
                },
                "required": ["city"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

system_prompt = "You are a helpful weather assistant."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the weather like in Paris today?"},
]

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [5]:
completion.model_dump()

{'id': 'chatcmpl-BYcZ9YKBKG0NedtqVbHSVBiK9MuMa',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_DUlhC11Wcp2HRqADRjNyIF2t',
      'function': {'arguments': '{"city":"Paris"}', 'name': 'get_weather'},
      'type': 'function'}]}}],
 'created': 1747591099,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_54eb4bd693',
 'usage': {'completion_tokens': 14,
  'prompt_tokens': 59,
  'total_tokens': 73,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [6]:
def call_func(name, args):
    print(name, args)
    if name == 'get_weather':
        return get_weather(**args)

In [7]:
import json

for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_func(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )


get_weather {'city': 'Paris'}


In [8]:
messages

[{'role': 'system', 'content': 'You are a helpful weather assistant.'},
 {'role': 'user', 'content': "What's the weather like in Paris today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DUlhC11Wcp2HRqADRjNyIF2t', function=Function(arguments='{"city":"Paris"}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_DUlhC11Wcp2HRqADRjNyIF2t',
  'content': '19.81'}]

In [34]:
from pydantic import BaseModel


class WeatherResponse(BaseModel):
    temprature: float
    llm_response: str


In [35]:
com = client.beta.chat.completions.parse(
    model='gpt-4o-mini',
    messages=messages,
    tools=tools,
    response_format=WeatherResponse
)

In [37]:
final_response = com.choices[0].message.parsed
final_response

WeatherResponse(temprature=19.81, llm_response='The current temperature in Paris today is approximately 19.8°C.')

In [26]:
fin = com.choices[0].message.parsed

In [30]:
com

ParsedChatCompletion[WeatherResponse](id='chatcmpl-BYcekxV7ep0VK5HitV14csdRq0Opj', choices=[ParsedChoice[WeatherResponse](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[WeatherResponse](content='{"city":"Paris"}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None, parsed=WeatherResponse(city='Paris')))], created=1747591446, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_54eb4bd693', usage=CompletionUsage(completion_tokens=10, prompt_tokens=116, total_tokens=126, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [28]:
com.choices[0].message.content

'{"city":"Paris"}'

### Prompt Chaining

In [1]:
import os

from openai import OpenAI
client = OpenAI()

In [4]:
from pydantic import BaseModel, Field
from typing import Optional

In [5]:
class EventExtraction(BaseModel):
    """first llm call: extract basic information"""

    desc: str = Field(description="raw description of event")
    is_calendar_event: bool = Field("wether this is calendar event")
    confidence_score: float = Field("confidence score between 0 and 1")


class EventConfirmation(BaseModel):
    """Third LLM call: Generate confirmation message"""

    confirmation_message: str = Field(
        description="Natural language confirmation message"
    )
    calendar_link: Optional[str] = Field(
        description="Generated calendar link if applicable"
    )
